In [1]:
pip install pandas numpy xgboost joblib

In [2]:
import joblib
import pandas as pd

# Load models
default_model = joblib.load("default_model_xgb.pkl")
default_features = joblib.load("model_features.pkl")
default_threshold = joblib.load("decision_threshold.pkl")

stress_model = joblib.load("emi_stress_model.pkl")
stress_features = joblib.load("emi_features.pkl")

health_model = joblib.load("financial_health_model.pkl")
health_features = joblib.load("health_model_features.pkl")

In [3]:
borrower = pd.DataFrame([{
    "age": 30,
    "MonthlyIncome": 55000,
    "NumberOfDependents": 2,
    "DebtRatio": 0.35,
    "NumberOfOpenCreditLinesAndLoans": 4,
    "NumberOfTimes90DaysLate": 0,
    "NumberRealEstateLoansOrLines": 1,
    "RevolvingUtilizationOfUnsecuredLines": 0.45,

    "loan_amnt": 250000,
    "term": 36,
    "int_rate": 13.5,
    "installment": 9000,
    "annual_inc": 660000,
    "dti": 20,
    "emp_length": 4
}])

In [4]:
d_input = borrower.reindex(columns=default_features, fill_value=0)
default_prob = default_model.predict_proba(d_input)[:,1][0]

In [5]:
s_input = borrower.reindex(columns=stress_features, fill_value=0)
stress_score = stress_model.predict(s_input)[0]

if stress_score < 30:
    stress_cat = "Low"
elif stress_score < 60:
    stress_cat = "Moderate"
elif stress_score < 80:
    stress_cat = "High"
else:
    stress_cat = "Severe"

In [6]:
borrower["default_probability"] = default_prob
borrower["emi_stress_score"] = stress_score

In [7]:
h_input = borrower.reindex(columns=health_features, fill_value=0)
health_score = health_model.predict(h_input)[0]

if health_score > 80:
    health_cat = "Excellent"
elif health_score > 60:
    health_cat = "Good"
elif health_score > 40:
    health_cat = "Risky"
else:
    health_cat = "Poor"

In [8]:
print("\n========= AI LOAN RISK REPORT =========\n")

print(f"Default Probability: {default_prob:.2f}")
print(f"EMI Stress Score: {stress_score:.1f} ({stress_cat})")
print(f"Financial Health Score: {health_score:.1f} ({health_cat})")

# Decision rules
if default_prob > 0.85:
    decision = "REJECT"
    reason = "Very high default risk"

elif health_score < 40:
    decision = "REJECT"
    reason = "Loan harmful for long-term finances"

elif stress_score > 80:
    decision = "APPROVE WITH CAUTION"
    reason = "Very high EMI burden"

elif default_prob > 0.6:
    decision = "APPROVE WITH CAUTION"
    reason = "Moderate default risk"

elif health_score > 75 and default_prob < 0.4:
    decision = "APPROVE"
    reason = "Financially safe borrower"

else:
    decision = "APPROVE WITH CAUTION"
    reason = "Balanced risk"

print("\nFINAL DECISION:", decision)
print("REASON:", reason)


========= AI LOAN RISK REPORT =========

Default Probability: 0.15
EMI Stress Score: 19.2 (Low)
Financial Health Score: 35.2 (Poor)

FINAL DECISION: REJECT
REASON: Loan harmful for long-term finances
